# Protein generator

### Import libraries

In [1]:
import urllib
import pandas as pd
from IPython.display import clear_output

### Function of retrieve conditional tags and sequences for a pfam_id from for uniprot

In [12]:
def get_sequence_and_conditional_tags(pfam_id):
    with urllib.request.urlopen(("https://www.uniprot.org/uniprot/?query=database%3A%28type%3Apfam+"+str(pfam_id)+"%29&precursor:no&columns=id,lineage(SUPERKINGDOM),lineage(KINGDOM),lineage(PHYLUM),lineage(CLASS),lineage(ORDER),linaege(FAMILY),lineage(GENUS),lineage(SPECIES),keywords,keyword-id,go(molecular%20function),go(cellular%20component),go(biological%20process),sequence&format=tab")) as f:
        query = f.read()
        query = str(query, 'utf-8')
    lines = query.split('\n')
    li_results = [x.split('\t') for x in lines]
    names = li_results[0]
    results = li_results[1:]
    return names, results

### Function to expand a column

In [13]:
def column_expander(df, key, splitkey):
    splits = df[key].str.split(splitkey, expand=True)
    new_columns = {}
    for split in range(len(splits.loc[0])):
        new_columns[split] = str(key)+"_"+str(split)
    splits.rename(columns = new_columns, inplace = True)
    df = pd.concat([df, splits], axis=1)
    return df 

### Download sequences, keywords and gene ontologies from uniprot

In [14]:
pfam_id = 'pf00123'
names, results = get_sequence_and_conditional_tags(pfam_id)

In [15]:
df = pd.DataFrame(results, columns=names)
df = df.dropna()

In [16]:
expand = {'Keyword ID':"; ",
          'Gene ontology (molecular function)':"; ",
          'Gene ontology (cellular component)':"; ",
          'Gene ontology (biological process)':"; ",
          'Keywords':";"}

for column in expand:
    df = column_expander(df, column, expand[column])
    df.drop(column, axis='columns', inplace=True)

In [17]:
seq = df['Sequence']

In [18]:
df.drop('Sequence', axis='columns', inplace=True)

In [19]:
df['Sequence'] = seq

In [20]:
df

,Entry,Taxonomic lineage (SUPERKINGDOM),Taxonomic lineage (KINGDOM),Taxonomic lineage (PHYLUM),Taxonomic lineage (CLASS),Taxonomic lineage (ORDER),Taxonomic lineage (GENUS),Taxonomic lineage (SPECIES),Keyword ID_0,Keyword ID_1,...,Keywords_5,Keywords_6,Keywords_7,Keywords_8,Keywords_9,Keywords_10,Keywords_11,Keywords_12,Keywords_13,Sequence
0,P09684,Eukaryota,Metazoa,Chordata,Actinopteri,Gadiformes (cods and others),Gadus,Gadus morhua (Atlantic cod),KW-0372,KW-0903,...,None,None,None,None,None,None,None,None,None,HSDAVFTDNYSRFRKQMAAKKYLNS
1,Q29W19,Eukaryota,Metazoa,Chordata,Mammalia,Artiodactyla,"Bos (oxen, cattle)",Bos taurus (Bovine),KW-0027,KW-0165,...,Secreted,Signal,None,None,None,None,None,None,None,MTMCSGARLALLVYGILMHSSVYGSPAASGLRFPGIRPENEVYDED...
2,P01280,Eukaryota,Metazoa,Chordata,Aves,Galliformes,Gallus,Gallus gallus (Chicken),KW-0027,KW-0372,...,None,None,None,None,None,None,None,None,None,HSDGLFTSEYSKMRGNAQVQKFIQNLM
3,P07217,Eukaryota,Metazoa,Chordata,Mammalia,Artiodactyla,Ovis,Ovis aries (Sheep),KW-0027,KW-0903,...,None,None,None,None,None,None,None,None,None,YADAIFTNSYRKILGQLSARKLLQDIMNRQQGERNQEQGAKVRL
4,Q60549,Eukaryota,Metazoa,Chordata,Mammalia,Rodentia,Mesocricetus,Mesocricetus auratus (Golden hamster),KW-0027,KW-0165,...,None,None,None,None,None,None,None,None,None,MPLWVFFVILTLTNGSHCSPSPSLPFRIRRYADAIFTSSYRKVLGQ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,A0A2R9AEX1,Eukaryota,Metazoa,Chordata,Mammalia,Primates,Pan (chimpanzees),Pan paniscus (Pygmy chimpanzee) (Bonobo),KW-0732,KW-1185,...,None,None,None,None,None,None,None,None,None,MDTRNKAQLLVLLTLLSVLFSQTSAWPLYRAPSALRLGDRIPFEGA...
2273,A0A2U3V0I5,Eukaryota,Metazoa,Chordata,Mammalia,Artiodactyla,Tursiops,Tursiops truncatus (Atlantic bottle-nosed dolp...,KW-0732,KW-1185,...,None,None,None,None,None,None,None,None,None,MLLWVFFLVTLTLSSGSHGSPPSQPLRMPRYADAIFTNSYRKVLAQ...
2274,A0A673V4D0,Eukaryota,Metazoa,Chordata,Mammalia,Carnivora,Suricata,Suricata suricatta (Meerkat),KW-0732,KW-1185,...,None,None,None,None,None,None,None,None,None,MTMCSGVRLALLVYGIIMHSSVYCSPAAAGLRFPGIRPDDEAYDED...
2275,A0A0D9RSY0,Eukaryota,Metazoa,Chordata,Mammalia,Primates,Chlorocebus,Chlorocebus sabaeus (Green monkey) (Cercopithe...,KW-0732,KW-1185,...,None,None,None,None,None,None,None,None,None,MKSIYFVAVLFVMLVQGSWQRSLQDTEEKPRSVSASQTDPLGDPDQ...


In [11]:
df.loc[1]

Entry                                                                          Q29W19
Taxonomic lineage (SUPERKINGDOM)                                            Eukaryota
Taxonomic lineage (KINGDOM)                                                   Metazoa
Taxonomic lineage (PHYLUM)                                                   Chordata
Taxonomic lineage (CLASS)                                                    Mammalia
                                                          ...                        
Keywords_10                                                                      None
Keywords_11                                                                      None
Keywords_12                                                                      None
Keywords_13                                                                      None
Sequence                            MTMCSGARLALLVYGILMHSSVYGSPAASGLRFPGIRPENEVYDED...
Name: 1, Length: 97, dtype: object